# Non-Quantized LLaMA Model

In [1]:
import huggingface_hub
from huggingface_hub import notebook_login

notebook_login()

In [2]:
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [3]:
model = "meta-llama/Llama-2-7b-chat-hf"

In [4]:
#PREPROCESS THE DATA
tokenizer = AutoTokenizer.from_pretrained(model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [5]:
#LET'S CREATE A PIPELINE

pipeline = transformers.pipeline("text-generation", model = model, torch_dtype = torch.float16, device_map = "auto")

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [10]:
sequences = pipeline(
    'Which is the lastest car that Lamborghini have launed in its V12 range? And also give details.\n',
    do_sample=True,
    top_k=10,
    top_p = 0.9,
    temperature = 0.5,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200, # can increase the length of sequence
)

In [11]:
for i in sequences:
  print(i['generated_text'])

Which is the lastest car that Lamborghini have launed in its V12 range? And also give details.
Lamborghini has launched the Huracan Performante, the latest version of its V12 supercar. The Huracan Performante is powered by a 5.2-liter V12 engine that produces 640 horsepower and 443 lb-ft of torque. It has a top speed of 202 mph and can accelerate from 0-62 mph in just 2.9 seconds.
The Performante features a number of upgrades over the standard Huracan, including a new aerodynamics package, a lightweight exhaust system, and a revised all-wheel-drive system. The car also features a new active rear steering system, which can adjust the angle of the rear wheels to improve handling and stability.


## Langchain

In [19]:
!pip install langchain

In [20]:
!pip install langchain_community

# Quantized Model

##HERE WE WILL LOAD THE QUANTIZED MODEL, SO FOR THAT WE USE "CTRANSFORMER"
##We can load in 3 formats
###1)ggml(Generative Graphical Mdoel)(CTransformers)
###2)gptq(Generative Post-Training quantization))
###3)gguf(GPT- Generated Unified Format))

##To better understand what's happening we need to dive into research papers to know the maths behind it.

In [21]:
!pip install ctransformers

In [22]:
from langchain.llms import CTransformers

In [23]:
#Let's load the model

quantized_llm = CTransformers(
    model = "TheBloke/Llama-2-7B-Chat-GGML",
    model_type = 'llama',
    config = {'max_new_tokens':256,
              'temperature':0.01}
    )

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [24]:
#Next we check with prediction

text = quantized_llm("What do you think about the job market in the future?")

In [26]:
print(text)



The job market is expected to change significantly in the coming years due to technological advancements and shifting societal trends. Here are some potential changes that could shape the future of work:

1. Automation and artificial intelligence: As technology continues to advance, more jobs may become automated, potentially leading to job displacement in certain sectors.
2. Remote work and flexible work arrangements: With the rise of remote work, there may be more flexibility in work schedules and locations, allowing for a better work-life balance and increased productivity.
3. Skills training and lifelong learning: As technologies evolve, workers may need to acquire new skills to remain relevant in the job market. Lifelong learning and upskilling may become more important.
4. Gig economy and non-traditional work arrangements: The gig economy and alternative work arrangements, such as freelancing or consulting, may continue to grow, offering more flexibility and autonomy for worker